### Load an Example Graph

In [ ]:
from ipyradiant import FileManager, PathLoader
import rdflib

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
rdf_graph = lw.graph
qres = lw.graph.query(
    """
    PREFIX planet: <https://swapi.co/resource/planet/>
    PREFIX human: <https://swapi.co/resource/human/>
    PREFIX film: <https://swapi.co/resource/film/>
    
    CONSTRUCT {
        ?s ?p ?o .
    }
    WHERE {
        ?s ?p ?o .
        
        VALUES (?s) {
            (human:1)
            (human:4)
#            (human:5)
#            (film:1)
#            (film:4)
#            (planet:1)
#            (planet:2)
        }
    }
    """
)

simple_graph = rdflib.graph.Graph().parse(data=qres.serialize(format="xml"))

In [ ]:
from ipyradiant.visualization import CytoscapeViewer

cv = CytoscapeViewer()
cv.graph = simple_graph
cv

### GEv2

In [ ]:
import ipycytoscape as cyto
import ipywidgets as W
import traitlets as T
import rdflib

from ipyradiant.visualization.explore.interactive_exploration import add_cyto_class, remove_cyto_class


# TODO need some type of warning that observers must be set every time the graph changes
class GEv2(W.Box):
    graph_view = T.Instance(CytoscapeViewer)
    rdf_graph = T.Instance(rdflib.graph.Graph)
    selected_node = T.Instance(cyto.Node, allow_none=True)
    
    _log = W.Output()
    
    def update_cytoscape_frontend(self):
        """A temporary workaround to trigger a frontend refresh"""
        self.graph_view.cytoscape_widget.graph.add_node(cyto.Node(data={"id": "random node"}))
        self.graph_view.cytoscape_widget.graph.remove_node_by_id("random node")
    
    def get_node(self, node: dict) -> cyto.Node:
        """This function is used to find a node given the id of a node copy"""

        for cyto_node in self.graph_view.cytoscape_widget.graph.nodes:
            if cyto_node.data["id"] == node["data"]["id"]:
                return cyto_node
        raise ValueError("Node not found in cytoscape.graph.nodes.")
    
    @_log.capture()
    def node_click_action(self, node: dict):
        # Have to get the object by dict reference
        cyto_node = self.get_node(node)
        
        if self.selected_node != cyto_node:
            if self.selected_node:
                self.selected_node.classes = remove_cyto_class(self.selected_node, "clicked")
            cyto_node.classes = add_cyto_class(cyto_node, "clicked")
        
        self.selected_node = cyto_node        
    
    @T.default("graph_view")
    def _create_graph_view(self):
        w = CytoscapeViewer()
        w.cytoscape_widget.on("node", "click", self.node_click_action)
        T.link((self, "rdf_graph"), (w, "graph"))
        return w
    
    @T.default("rdf_graph")
    def _create_rdf_graph(self):
        return rdflib.Graph()
    
    @T.default("selected_node")
    def _create_default_selected_node(self):
        return None
    
    @T.validate("children")
    def validate_children(self, proposal):
        """
        Validate method for default children.
        This is necessary because @T.default does not work on children.
        """
        children = proposal.value
        if not children:
            children = (self.graph_view,)
        return children

In [ ]:
gev2 = GEv2()
gev2.rdf_graph = simple_graph
gev2